# Implementing Decision Trees 

### In this code we are going to implement Decision Trees manually and for that we will use Iris Dataset of SkLearn  

# ENTROPY:
### In this im going to use Entropy and Information Gain
###             FORMULA : - SUM of (p*log2(p))
### where p = probability of happening that attribute
### Entropy is used to calculate the accuracy of one of the class of data 

# INFORMATION GAIN
### IG is basically used to calculate that which question should be asked now to increase our accuracy at the current moment
### FORMULA : E(Parent Node) - (Weighted Average)*(E(Child Node))
### E --> Entropy 

In [21]:
import pandas as pd
import numpy as np
from sklearn import datasets

In [22]:
iris_data = datasets.load_iris()
iris_data

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [23]:
data = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
data['variety'] = iris_data.target
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


# Cleaning The Data

In [24]:
data.isna().sum()
# Clearly no null value is present here in the entire dataset so no need to drop any values

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
variety              0
dtype: int64

# Building Decision Tree Classifier

### Node Class

In [25]:
class Node():
# Below is the Constructor for the node    
    def __init__(self, feature_index = None, threshold = None, left = None, right = None, info_gain = None, value = None):
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

        # for leaf node as at that node we are only concerned with the majority value
        self.value = value

### Tree Class

In [26]:
class DecisionTree():
# Below is the constructor of the tree    
    def __init__(self, min_samples_split = 2, max_depth = 2):
        
        # initialize the root of the tree
        self.root = None

        # stopping conditions
        self.min_samples_split = min_samples_split # when the number of samples exceed the min number of samples
        self.max_depth = max_depth # When we have reached a particular depth after that we dont want our tree to build further

# Below there is the recursive code for tree building 
    def build_tree(self, dataset, curr_depth):
        # dataset here refers to the data which would be given to us 
        X,Y = dataset[:,:-1], dataset[:,-1] # X refers to the data in which the last column is dropped (in this case variety column)
        num_samples, num_features = np.shape(X)
        
        # split untill stopping conditions are met 
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"] > 0:
                # Calling recursion to the left
                left_subtree = self.build_tree(best_split['dataset_left'], curr_depth+1)
                # Calling recursion to the right 
                right_subtree = self.build_tree(best_split['dataset_right'], curr_depth+1)
                # Return Decision Node
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])
        
        # Compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # Return leaf Node
        return Node(value = leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        # Function to find the best split

        # Dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf") # It means -INT_MAX

        # Loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:,feature_index]
            possible_thresholds = np.unique(feature_values)
            
            # looping all over the feature values present in the data
            for threshold in possible_thresholds:
                # getting current split
                dataset_left, dataset_right = self.split(dataset,feature_index,threshold)
                # check if childs are not null
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:,-1], dataset_left[:,-1], dataset_right[:,-1]
                    # Compute IG
                    curr_info_gain = self.information_gain(y,left_y,right_y,"gini")
                    # Update the best split if needed
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index 
                        best_split["threshold"] =  threshold
                        best_split["dataset_left"] =  dataset_left
                        best_split["dataset_right"] = dataset_right 
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain 
        
        # Returning the best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        # Function to split data

        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left,dataset_right
    
    # Function to calculate information gain
    def information_gain(self, parent, l_child, r_child, mode = "entropy"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode == "gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        
        return gain
    
    # Function to compute Entropy
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy = entropy + (-p_cls * np.log2(p_cls))
        return entropy
    
    # Function to compute Gini Index
    def gini(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls] / len(y))
            gini = gini + p_cls**2
        return 1-gini
    
    # Function to compute the last leaf node value
    def calculate_leaf_value(self, Y):
        Y = list(y)
        return max(Y, key=Y.count)
    
    # Function to print the decision tree
    def print_tree(self, tree = None, indent = " "):
        if not tree:
            tree = self.root
        
        if tree.value is not None:
            print(tree.value)
        else:
            print("X " + str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" %(indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    # Function to train the tree
    def fit(self, X, Y):
        dataset = np.concatenate((X,Y), axis=1)
        self.root = self.build_tree(dataset,curr_depth)

    # Fuction to predict new dataset
    def predict(self, X):
        predictions = [self.make_predicitons(x,self.root) for x in X]
        return predictions
    
    # Function to predict a single data point
    def make_prediction(self, x, tree):
        if tree.value != None : return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

### Train-Test Split

In [27]:
X = data.iloc[:,:-1].values
Y = data.iloc[:,-1].values.reshape(-1,1)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=2,random_state=41)

### Fit the Model

In [28]:
classifier = DecisionTree(min_samples_split=3,max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

NameError: name 'curr_depth' is not defined

### Test the Model

In [ ]:
Y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)